In [ ]:
%pylab inline

In [ ]:
import numpy as np
import matplotlib.pyplot as pt
import matplotlib.image as img
from PIL import Image
from skimage import color
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.cluster import KMeans
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn import cross_validation
from sklearn import linear_model, datasets
#import pydot
from sklearn import tree
from sklearn.externals.six import StringIO

In [ ]:
#Returns dictionary of labels; key is label as string, value is a unique index
def get_category_labels_as_dict():
    category_dict = {}
    num_category = 0
    with open('./CS5785-final-data/train.txt') as f:
        lines = f.read().splitlines()
        for line in lines:
            tokens = line.split(' ')
            if not category_dict.has_key(tokens[1]):
                category_dict[tokens[1]] = num_category
                num_category += 1
    return category_dict

In [ ]:
#Returns labels of training set as an array of strings
def get_train_labels():
    train_labels = []
    with open('./CS5785-final-data/train.txt') as f:
        lines = f.read().splitlines()
        for line in lines:
            tokens = line.split(' ')
            train_labels.append(tokens[1])
    return train_labels

In [ ]:
#Returns train labels as numpy array, each label is unique index; params: dictionary of label to unique index, array of labels as strings
#Usage: get_train_labels_as_unique_indices(get_category_labels_as_dict(), get_train_labels())
def get_train_labels_as_unique_indices(label_dictionary, train_label_array):
    train_labels_as_index = []
    for i in range(len(train_label_array)):
        label_name = train_label_array[i]
        index = label_dictionary[label_name]
        train_labels_as_index.append(index)
    return np.array(train_labels_as_index)

Representation method 1: Boolean 102 keywords

In [ ]:
#Returns attributes of train data as numpy array
def get_train_attributes():
    feature_vectors = []
    with open('./CS5785-final-data/attributes_train.txt') as f:
        lines = f.read().splitlines()
        for line in lines:
            tokens = line.split(' ')
            values = tokens[1].split(',')
            fv = []
            for value in values:
                fv.append(int(value))
            feature_vectors.append(fv)
    return np.array(feature_vectors)

In [ ]:
#Returns attributes of test data as numpy array
def get_test_attributes():
    feature_vectors = []
    with open('./CS5785-final-data/attributes_test.txt') as f:
        lines = f.read().splitlines()
        for line in lines:
            tokens = line.split(' ')
            values = tokens[1].split(',')
            fv = []
            for value in values:
                fv.append(int(value))
            feature_vectors.append(fv)
    return np.array(feature_vectors)

Representation method 2: SIFT array

In [ ]:
#Returns SIFT attributes of training data as numpy array
def get_train_SIFT():
    return np.load('./CS5785-final-data/SIFTBoW_train.npy')

In [ ]:
#Returns SIFT attributes of test data as numpy array
def get_test_SIFT():
    return np.load('./CS5785-final-data/SIFTBoW_test.npy')

Representation method 3: ALEX array

In [ ]:
#Returns ALEX attributes of train data as numpy array
def get_train_ALEX():
    return np.load('./CS5785-final-data/alexnet_feat_train.npy')

In [ ]:
#Returns ALEX attributes of test data as numpy array
def get_test_ALEX():
    return np.load('./CS5785-final-data/alexnet_feat_test.npy')

In [ ]:
def get_crossvalidation(X_train, Y_train, n_fold, model):
    cv = cross_validation.KFold(len(X_train), n_folds = n_fold)
    
    results = []
    for traincv, testcv in cv:
        results.append(cross_validation.cross_val_score(model, X_train[traincv], Y_train[traincv]))
    
    return np.array(results).mean()    

In [ ]:
categories = get_category_labels_as_dict()
print len(categories)

In [ ]:
category_dictionary = get_category_labels_as_dict()
Y_train_text = get_train_labels()

In [ ]:
X_train = get_train_attributes()
Y_train = get_train_labels_as_unique_indices(category_dictionary, Y_train_text)
X_test = get_test_attributes()

In [ ]:
lr = linear_model.LogisticRegression()
Y_predicted_1 = lr.fit(X_train, Y_train).predict(X_test)

In [ ]:
X_train = get_train_ALEX()
X_test = get_test_ALEX()
#Y_predicted_2 = lr.fit(X_train, Y_train).predict(X_test)

In [ ]:
X_train = get_train_SIFT()
X_test = get_test_SIFT()
#Y_predicted_3 = lr.fit(X_train, Y_train).predict(X_test)

In [ ]:
Y_predicted = []
for i in range (0, len(Y_predicted_1)):
    vote = []
    vote.append(Y_predicted_1[i])
    vote.append(Y_predicted_2[i])
    vote.append(Y_predicted_3[i])
    if(vote.count(vote[1])>=2):
        Y_predicted.append(vote[1])
    else: Y_predicted.append(vote[0])
Y_predicted = np.array(Y_predicted)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier()
Y_predicted_RF = RF.fit(X_train, Y_train).predict(X_test)

In [ ]:
category = {}
for i in categories:
    category[categories[i]] = i

In [ ]:
Y_test = []
with open('./CS5785-final-data/test.txt') as f:
    lines = f.read().splitlines()
    for line in lines:
        Y_test.append(line)

In [ ]:
import csv

with open('submission_1.csv', 'w') as csvfile:
    fieldnames = ['ID', 'category']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for i in range (0, 1000):
        writer.writerow({'ID': Y_test[i], 'category': category[Y_predicted_svm[i]]})


In [ ]:
RF = RandomForestClassifier()
lr = linear_model.LogisticRegression()
gnb = GaussianNB()
bnb = BernoulliNB()
knnc = KNeighborsClassifier()
knnr = KNeighborsRegressor()

print get_crossvalidation(X_train, Y_train, 3, lr)
print get_crossvalidation(X_train, Y_train, 3, RF)
print get_crossvalidation(X_train, Y_train, 3, gnb)
print get_crossvalidation(X_train, Y_train, 3, bnb)
print get_crossvalidation(X_train, Y_train, 3, knn)